# CIFAR-10图像分类

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

device=torch.device('cuda:7' if torch.cuda.is_available() else 'cpu')
print(f'using device:{device}')
batch_size=256
learning_rate=0.01
num_epochs=10

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914,0.4822,0.4465),(0.2023,0.1994,0.2010))
])#(R,G,B)的均值和标准差

train_set=torchvision.datasets.CIFAR10(root='./data',train=True,download=True,transform=transform)
trian_loader=torch.utils.data.Dataloader(train_set,batch_size=batch_size,shuffle=True)
test_set=torchvision.datasets.CIFAR10(root='./data',train=False,download=True,transform=transform)
test_loader=torch.utils.data.Dataloader(test_set,batch_size=batch_size,shuffle=False)


